In [1]:
import sys
import numpy as np
import pandas as pd
from matplotlib.pyplot import imread
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

Using TensorFlow backend.


In [2]:
train_folder = "/home/rohit/Downloads/hand_copy/train/"
val_folder = '/home/rohit/Downloads/hand_copy/val/'
save_path = '/home/rohit/Downloads/hand_copy/data/'

In [3]:
def loadimgs(path,n = 0):
    '''
    path => Path of train directory or test directory
    '''
    X=[]
    y = []
    cat_dict = {}
    lang_dict = {}
    curr_y = n
    # we load every alphabet seperately so we can isolate them later
    for alphabet in os.listdir(path):
        print("loading alphabet: " + alphabet)
        lang_dict[alphabet] = [curr_y,None]
        alphabet_path = os.path.join(path,alphabet)
        # every letter/category has it's own column in the array, so  load seperately
        for letter in os.listdir(alphabet_path):
            cat_dict[curr_y] = (alphabet, letter)
            category_images=[]
            letter_path = os.path.join(alphabet_path, letter)
            # read all the images in the current category
            for filename in os.listdir(letter_path):
                image_path = os.path.join(letter_path, filename)
                image = imread(image_path)
                category_images.append(image)
                y.append(curr_y)
            try:
                X.append(np.stack(category_images))
            # edge case  - last one
            except ValueError as e:
                print(e)
                print("error - category_images:", category_images)
            curr_y += 1
            lang_dict[alphabet][1] = curr_y - 1
    y = np.vstack(y)
    X = np.stack(X)
    return X,y,lang_dict


In [4]:
X,y,c=loadimgs(train_folder)

loading alphabet: B
loading alphabet: L
loading alphabet: C
loading alphabet: R
loading alphabet: D
loading alphabet: I
loading alphabet: A
loading alphabet: J
loading alphabet: K
loading alphabet: Q
loading alphabet: M
loading alphabet: N
loading alphabet: E
loading alphabet: G
loading alphabet: H
loading alphabet: O
loading alphabet: P
loading alphabet: F


In [5]:
with open(os.path.join(save_path,"train.pickle"), "wb") as f:
    pickle.dump((X,c),f)

In [6]:
Xval,yval,cval=loadimgs(val_folder)

loading alphabet: S
loading alphabet: Z
loading alphabet: nothing
loading alphabet: U
loading alphabet: space
loading alphabet: Y
loading alphabet: X
loading alphabet: T
loading alphabet: W


In [7]:
with open(os.path.join(save_path,"val.pickle"), "wb") as f:
    pickle.dump((Xval,cval),f)

In [8]:
from keras import models
from keras import layers
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = models.Sequential()
    model.add(layers.Conv2D(64, (10,10), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(128, (7,7), activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(128, (4,4), activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(256, (4,4), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='sigmoid'))
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid')(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [9]:
model = get_siamese_model((105, 105, 1))
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 4096)         38947648    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 4096)         0           sequential_1[1][0]         

In [10]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

In [11]:
with open(os.path.join(save_path, "train.pickle"), "rb") as f:
    (Xtrain, train_classes) = pickle.load(f)
    
print("Training alphabets: \n")
print(list(train_classes.keys()))

Training alphabets: 

['B', 'L', 'C', 'R', 'D', 'I', 'A', 'J', 'K', 'Q', 'M', 'N', 'E', 'G', 'H', 'O', 'P', 'F']


In [12]:
with open(os.path.join(save_path, "val.pickle"), "rb") as f:
    (Xval, val_classes) = pickle.load(f)

print("Validation alphabets:", end="\n\n")
print(list(val_classes.keys()))

Validation alphabets:

['S', 'Z', 'nothing', 'U', 'space', 'Y', 'X', 'T', 'W']


In [13]:
def get_batch(batch_size,s="train"):
    """Create batch of n pairs, half same class, half different class"""
    if s == 'train':
        X = Xtrain
        categories = train_classes
    else:
        X = Xval
        categories = val_classes
    n_classes, n_examples, w, h = X.shape

    # randomly sample several classes to use in the batch
    categories = rng.choice(n_classes,size=(batch_size,),replace=False)
    
    # initialize 2 empty arrays for the input image batch
    pairs=[np.zeros((batch_size, h, w,1)) for i in range(2)]
    
    # initialize vector for the targets
    targets=np.zeros((batch_size,))
    
    # make one half of it '1's, so 2nd half of batch has same class
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = rng.randint(0, n_examples)
        pairs[0][i,:,:,:] = X[category, idx_1].reshape(w, h, 1)
        idx_2 = rng.randint(0, n_examples)
        
        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            # add a random number to the category modulo n classes to ensure 2nd image has a different category
            category_2 = (category + rng.randint(1,n_classes)) % n_classes
        
        pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(w, h,1)
    
    return pairs, targets

In [14]:
def generate(batch_size, s="train"):
    """a generator for batches, so model.fit_generator can be used. """
    while True:
        pairs, targets = get_batch(batch_size,s)
        yield (pairs, targets)

In [15]:
def make_oneshot_task(N, s="val", language=None):
    """Create pairs of test image, support set for testing N way one-shot learning. """
    if s == 'train':
        X = Xtrain
        categories = train_classes
    else:
        X = Xval
        categories = val_classes
    n_classes, n_examples, w, h = X.shape
    
    indices = rng.randint(0, n_examples,size=(N,))
    if language is not None: # if language is specified, select characters for that language
        low, high = categories[language]
        if N > high - low:
            raise ValueError("This language ({}) has less than {} letters".format(language, N))
        categories = rng.choice(range(low,high),size=(N,),replace=False)

    else: # if no language specified just pick a bunch of random letters
        categories = rng.choice(range(n_classes),size=(N,),replace=False)            
    true_category = categories[0]
    ex1, ex2 = rng.choice(n_examples,replace=False,size=(2,))
    test_image = np.asarray([X[true_category,ex1,:,:]]*N).reshape(N, w, h,1)
    support_set = X[categories,indices,:,:]
    support_set[0,:,:] = X[true_category,ex2]
    support_set = support_set.reshape(N, w, h,1)
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image,support_set]

    return pairs, targets

In [16]:
def test_oneshot(model, N, k, s = "val", verbose = 0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N,s)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    return percent_correct

In [17]:
evaluate_every = 200 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 2000 # No. of training iterations
N_way = 20 # how many classes for testing one-shot tasks
n_val = 250 # how many one-shot tasks to validate on
best = -1

In [18]:
model_path = '/home/rohit/Downloads/hand_copy/weights/'

In [20]:
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter+1):
    print("Starting ", i, "iteration")
    (inputs,targets) = get_batch(batch_size)
    loss = model.train_on_batch(inputs, targets)
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
        print("Train Loss: {0}".format(loss)) 
        val_acc = test_oneshot(model, N_way, n_val, verbose=True)
        model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            best = val_acc

Starting training process!
-------------------------------------
Starting  1 iteration
Starting  2 iteration
Starting  3 iteration
Starting  4 iteration
Starting  5 iteration
Starting  6 iteration
Starting  7 iteration
Starting  8 iteration
Starting  9 iteration
Starting  10 iteration
Starting  11 iteration
Starting  12 iteration
Starting  13 iteration
Starting  14 iteration
Starting  15 iteration
Starting  16 iteration
Starting  17 iteration
Starting  18 iteration
Starting  19 iteration
Starting  20 iteration
Starting  21 iteration
Starting  22 iteration
Starting  23 iteration
Starting  24 iteration
Starting  25 iteration
Starting  26 iteration
Starting  27 iteration
Starting  28 iteration
Starting  29 iteration
Starting  30 iteration
Starting  31 iteration
Starting  32 iteration
Starting  33 iteration
Starting  34 iteration
Starting  35 iteration
Starting  36 iteration
Starting  37 iteration
Starting  38 iteration
Starting  39 iteration
Starting  40 iteration
Starting  41 iteration
S

Starting  334 iteration
Starting  335 iteration
Starting  336 iteration
Starting  337 iteration
Starting  338 iteration
Starting  339 iteration
Starting  340 iteration
Starting  341 iteration
Starting  342 iteration
Starting  343 iteration
Starting  344 iteration
Starting  345 iteration
Starting  346 iteration
Starting  347 iteration
Starting  348 iteration
Starting  349 iteration
Starting  350 iteration
Starting  351 iteration
Starting  352 iteration
Starting  353 iteration
Starting  354 iteration
Starting  355 iteration
Starting  356 iteration
Starting  357 iteration
Starting  358 iteration
Starting  359 iteration
Starting  360 iteration
Starting  361 iteration
Starting  362 iteration
Starting  363 iteration
Starting  364 iteration
Starting  365 iteration
Starting  366 iteration
Starting  367 iteration
Starting  368 iteration
Starting  369 iteration
Starting  370 iteration
Starting  371 iteration
Starting  372 iteration
Starting  373 iteration
Starting  374 iteration
Starting  375 it

Starting  655 iteration
Starting  656 iteration
Starting  657 iteration
Starting  658 iteration
Starting  659 iteration
Starting  660 iteration
Starting  661 iteration
Starting  662 iteration
Starting  663 iteration
Starting  664 iteration
Starting  665 iteration
Starting  666 iteration
Starting  667 iteration
Starting  668 iteration
Starting  669 iteration
Starting  670 iteration
Starting  671 iteration
Starting  672 iteration
Starting  673 iteration
Starting  674 iteration
Starting  675 iteration
Starting  676 iteration
Starting  677 iteration
Starting  678 iteration
Starting  679 iteration
Starting  680 iteration
Starting  681 iteration
Starting  682 iteration
Starting  683 iteration
Starting  684 iteration
Starting  685 iteration
Starting  686 iteration
Starting  687 iteration
Starting  688 iteration
Starting  689 iteration
Starting  690 iteration
Starting  691 iteration
Starting  692 iteration
Starting  693 iteration
Starting  694 iteration
Starting  695 iteration
Starting  696 it

Starting  987 iteration
Starting  988 iteration
Starting  989 iteration
Starting  990 iteration
Starting  991 iteration
Starting  992 iteration
Starting  993 iteration
Starting  994 iteration
Starting  995 iteration
Starting  996 iteration
Starting  997 iteration
Starting  998 iteration
Starting  999 iteration
Starting  1000 iteration

 ------------- 

Time for 1000 iterations: 40.65287698904673 mins
Train Loss: 0.04417704790830612
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 47.6% 20 way one-shot learning accuracy 

Current best: 47.6, previous best: 34.0
Starting  1001 iteration
Starting  1002 iteration
Starting  1003 iteration
Starting  1004 iteration
Starting  1005 iteration
Starting  1006 iteration
Starting  1007 iteration
Starting  1008 iteration
Starting  1009 iteration
Starting  1010 iteration
Starting  1011 iteration
Starting  1012 iteration
Starting  1013 iteration
Starting  1014 iteration
Starting  1015 iteration
Starting  1016 iterat

Starting  1296 iteration
Starting  1297 iteration
Starting  1298 iteration
Starting  1299 iteration
Starting  1300 iteration
Starting  1301 iteration
Starting  1302 iteration
Starting  1303 iteration
Starting  1304 iteration
Starting  1305 iteration
Starting  1306 iteration
Starting  1307 iteration
Starting  1308 iteration
Starting  1309 iteration
Starting  1310 iteration
Starting  1311 iteration
Starting  1312 iteration
Starting  1313 iteration
Starting  1314 iteration
Starting  1315 iteration
Starting  1316 iteration
Starting  1317 iteration
Starting  1318 iteration
Starting  1319 iteration
Starting  1320 iteration
Starting  1321 iteration
Starting  1322 iteration
Starting  1323 iteration
Starting  1324 iteration
Starting  1325 iteration
Starting  1326 iteration
Starting  1327 iteration
Starting  1328 iteration
Starting  1329 iteration
Starting  1330 iteration
Starting  1331 iteration
Starting  1332 iteration
Starting  1333 iteration
Starting  1334 iteration
Starting  1335 iteration


Starting  1604 iteration
Starting  1605 iteration
Starting  1606 iteration
Starting  1607 iteration
Starting  1608 iteration
Starting  1609 iteration
Starting  1610 iteration
Starting  1611 iteration
Starting  1612 iteration
Starting  1613 iteration
Starting  1614 iteration
Starting  1615 iteration
Starting  1616 iteration
Starting  1617 iteration
Starting  1618 iteration
Starting  1619 iteration
Starting  1620 iteration
Starting  1621 iteration
Starting  1622 iteration
Starting  1623 iteration
Starting  1624 iteration
Starting  1625 iteration
Starting  1626 iteration
Starting  1627 iteration
Starting  1628 iteration
Starting  1629 iteration
Starting  1630 iteration
Starting  1631 iteration
Starting  1632 iteration
Starting  1633 iteration
Starting  1634 iteration
Starting  1635 iteration
Starting  1636 iteration
Starting  1637 iteration
Starting  1638 iteration
Starting  1639 iteration
Starting  1640 iteration
Starting  1641 iteration
Starting  1642 iteration
Starting  1643 iteration


Starting  1922 iteration
Starting  1923 iteration
Starting  1924 iteration
Starting  1925 iteration
Starting  1926 iteration
Starting  1927 iteration
Starting  1928 iteration
Starting  1929 iteration
Starting  1930 iteration
Starting  1931 iteration
Starting  1932 iteration
Starting  1933 iteration
Starting  1934 iteration
Starting  1935 iteration
Starting  1936 iteration
Starting  1937 iteration
Starting  1938 iteration
Starting  1939 iteration
Starting  1940 iteration
Starting  1941 iteration
Starting  1942 iteration
Starting  1943 iteration
Starting  1944 iteration
Starting  1945 iteration
Starting  1946 iteration
Starting  1947 iteration
Starting  1948 iteration
Starting  1949 iteration
Starting  1950 iteration
Starting  1951 iteration
Starting  1952 iteration
Starting  1953 iteration
Starting  1954 iteration
Starting  1955 iteration
Starting  1956 iteration
Starting  1957 iteration
Starting  1958 iteration
Starting  1959 iteration
Starting  1960 iteration
Starting  1961 iteration
